# Tools


- Author: [Jaeho Kim](https://github.com/Jae-hoya)
- Design: []()
- Peer Review:
- This is a part of [LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial)

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain-academy/blob/main/module-4/sub-graph.ipynb) [![Open in LangChain Academy](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/58239937-lesson-2-sub-graphs)

## OverView

A tool is an interface that allows agents, chains, or LLMs to interact with the external world.

LangChain provides built-in tools that are easy to use, and it also enables users to easily build custom tools.

**You can find the list of tools integrated into LangChain at the link below.**

- [List of Tools Integrated with LangChain](https://python.langchain.com/docs/integrations/tools/)

### Table of Contents

- [Overview](#overview)
- [Environment Setup](#environment-setup)
- [Built-in tools](#built-in-tools)
- [Python REPL Tool](#python-repl-tool)
- [Search API Tool(Tavily)](#search-api-tooltavily)
- [Image Generation Tool (DALL-E)](#image-generation-tool-dall-e)
- [Custom Tools](#custom-tools)
### References

- [LangChain: List of Integrated Tools](https://python.langchain.com/docs/integrations/tools/)
- [LangChain Tools/Toolkits](https://python.langchain.com/docs/integrations/tools/)
- [LangChain: PythonREPLTool](https://python.langchain.com/docs/integrations/tools/python/)
- [LangChain: Tavily Search](https://python.langchain.com/docs/integrations/tools/tavily_search/)
---

## Environment Setup

Set up the environment. You may refer to [Environment Setup](https://wikidocs.net/257836) for more details.

**[Note]**
- `langchain-opentutorial` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials. 
- You can checkout the [`langchain-opentutorial`](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.

In [1]:
%%capture --no-stderr
%pip install langchain-opentutorial

In [2]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain",
        "langchain_core",
        "langchain_openai",
        "langchain_experimental",
        "tavily-python",
        "feedparser",
    ],
    verbose=False,
    upgrade=False,
)

In [3]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        "OPENAI_API_KEY": "",
        "TAVILY_API_KEY": "",
        "LANGCHAIN_API_KEY": "",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "01-Tools",
    }
)

Environment variables have been set successfully.


Environment variables have been set successfully.
You can alternatively set API keys, such as `OPENAI_API_KEY` in a `.env` file and load them.

[Note] This is not necessary if you've already set the required API keys in previous steps.

In [4]:
# Load API keys from .env file
from dotenv import load_dotenv

load_dotenv(override=True)

True

**Note**

When using tools, warning messages may be displayed.

For example, there are security warnings in the **REPL** environment.
`PythonREPLTool` is a tool for executing Python code and can potentially execute commands (e.g., file system access, network requests) that may pose security risks.
In such cases, LangChain or Python itself may output warning messages.

To ignore these warning messages, you can use the following code:

In [5]:
import warnings

# Ignore warning messages.
warnings.filterwarnings("ignore")

## Built-in tools

You can use pre-defined tools and toolkits provided by LangChain.

A tool refers to a single utility, while a toolkit combines multiple tools into a single unit for use.

You can find the relevant tools at the link below.

**Note**
- [LangChain Tools/Toolkits](https://python.langchain.com/docs/integrations/tools/)

## Python REPL Tool

This tool provides a class for executing Python code in a **REPL (Read-Eval-Print Loop)** environment.
- [PythonREPLTool](https://python.langchain.com/docs/integrations/tools/python/)

**Description**

- Provides a Python shell environment.
- Executes valid Python commands as input.
- Use the `print(...)` function to view results.

**Key Features**

- sanitize_input: Option to sanitize input (default: True)
- python_repl: Instance of **PythonREPL** (default: executed in the global scope)

**Usage**

- Create an instance of `PythonREPLTool `.
- Execute Python code using the `run` , `arun` , or `invoke` methods.

**Input Sanitization**

- Removes unnecessary spaces, backticks, the keyword "python," and other extraneous elements from the input string.

In [6]:
from langchain_experimental.tools import PythonREPLTool

# Creates a tool for executing Python code.
python_tool = PythonREPLTool()

In [7]:
# Executes Python code and returns the results.
print(python_tool.invoke("print(100 + 200)"))

Python REPL can execute arbitrary code. Use with caution.


300



Below is an example of requesting an LLM to write Python code and returning the results.

**Workflow Summary**
1. Request the LLM model to write Python code for a specific task.
2. Execute the generated code to obtain the results.
3. Output the results.

**Note**

I recommend using a model equivalent to or higher than GPT-4.

In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda


# A function that executes Python code, outputs intermediate steps, and returns the tool execution results.
def print_and_execute(code, debug=True):
    if debug:
        print("CODE:")
        print(code)
    return python_tool.invoke(code)


# A prompt requesting Python code to be written.
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are Raymond Hetting, an expert python programmer, well versed in meta-programming and elegant, concise and short but well documented code. You follow the PEP8 style guide. "
            "Return only the code, no intro, no explanation, no chatty, no markdown, no code block, no nothing. Just the code.",
        ),
        ("human", "{input}"),
    ]
)
# Create LLM model.
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# Create a chain using the prompt and the LLM model.
chain = prompt | llm | StrOutputParser() | RunnableLambda(print_and_execute)

In [9]:
# Outputting the results.
print(chain.invoke("Write code to generate Powerball numbers."))

CODE:
import random

def generate_powerball():
    main_numbers = sorted(random.sample(range(1, 70), 5))
    powerball = random.randint(1, 26)
    return main_numbers, powerball

# Example usage
main_numbers, powerball = generate_powerball()
print("Main numbers:", main_numbers)
print("Powerball:", powerball)
Main numbers: [7, 17, 18, 51, 52]
Powerball: 16



## Search API Tool(Tavily)

This is a tool that implements a search function using the `Tavily` Search API. It provides two main classes: `TavilySearchResults` and `TavilyAnswer` .

**API Key Issuance URL**
- https://app.tavily.com/

Set the issued API key as an environment variable.

For example, configure the .env file as follows:

```
TAVILY_API_KEY=tvly-abcdefghijklmnopqrstuvwxyz
```

### TavilySearchResults

**Description**
- Queries the Tavily Search API and returns results in JSON format.
- A search engine optimized for comprehensive, accurate, and reliable results.
- Useful for answering questions about current events.

**Key Parameters**
- `max_results` (int): Maximum number of search results to return (default: 5).
- `search_depth` (str): Search depth ("basic" or "advanced").
- `include_domains` (List[str]): List of domains to include in search results.
- `exclude_domains` (List[str]): List of domains to exclude from search results.
- `include_answer` (bool): Whether to include a short answer to the original query.
- `include_raw_content` (bool): Whether to include refined HTML content from each site.
- `include_images` (bool): Whether to include a list of images related to the query.

**Return Value**
- A JSON-formatted string containing the search results (url, content).

In [10]:
from langchain_community.tools.tavily_search import TavilySearchResults

# Create tool.
tool = TavilySearchResults(
    max_results=6,
    include_answer=True,
    include_raw_content=True,
    # include_images=True,
    # search_depth="advanced", # or "basic"
    include_domains=["github.io"],
    # exclude_domains = []
)

In [11]:
# Execute tool.
tool.invoke({"query": "What is Langchain Tools?"})

[{'url': 'https://stonefishy.github.io/2024/11/12/introduction-to-langchain-make-ai-smarter-and-easy-to-use/',
  'content': 'LangChain makes it easier by offering ready-made building blocks to connect these models to other tools, data, and even databases. Think of LangChain like a set of Lego blocks that you can use to build cool things with AI.'},
 {'url': 'https://kirenz.github.io/lab-langchain-functions/slides/05_tools_routing.html',
  'content': 'Langchain Functions - Tools and Routing from langchain.agents import tool from langchain.tools.render import format_tool_to_openai_function format_tool_to_openai_function(get_current_temperature) {‘name’: ‘get_current_temperature’, ‘description’: ‘get_current_temperature(latitude: float, longitude: float) -> dict - Fetch current temperature for given coordinates.’, ‘parameters’: {‘title’: ‘OpenMeteoInput’, ‘type’: ‘object’, ‘properties’: {‘latitude’: {‘title’: ‘Latitude’, ‘description’: ‘Latitude of the location to fetch weather data for’,

## Image Generation Tool (DALL-E)

- `DallEAPIWrapper Class` : A wrapper for OpenAI's DALL-E image generator.

This tool allows easy integration of the DALL-E API to implement text-based image generation functionality. With various configuration options, it can be utilized as a flexible and powerful image generation tool.

**Key Properties**

- `model` : The name of the DALL-E model to use ( **dall-e-2**, **dall-e-3** ).

- `n` : Number of images to generate (default: 1).

- `size` : Size of the generated image:
  - "dall-e-2": "1024x1024", "512x512", "256x256"
  - "dall-e-3": "1024x1024", "1792x1024", "1024x1792"

- `style` : Style of the generated image ( **natural** , **vivid** ).

- `quality` : Quality of the generated image ( **standard**, **hd** ).

- `max_retries` : Maximum number of retries for generation.

**Key Features**
- Generates images based on text descriptions using the DALL-E API.

**Workflow Summary**

The following is an example of generating images using the `DALL-E` Image Generator.

This time, we will use the `DallEAPIWrapper` to generate images.

The input prompt will request the LLM model to write a prompt for generating images.

In [12]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# Initialize the ChatOpenAI model
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.9, max_tokens=1000)

# Define a prompt template for DALL-E image generation
prompt = PromptTemplate.from_template(
    "Generate a detailed IMAGE GENERATION prompt for DALL-E based on the following description. "
    "Return only the prompt, no intro, no explanation, no chatty, no markdown, no code block, no nothing. Just the prompt"
    "Output should be less than 1000 characters. Write in English only."
    "Image Description: \n{image_desc}",
)

# Create a chain connecting the prompt, LLM, and output parser
chain = prompt | llm | StrOutputParser()

# Execute the chain
image_prompt = chain.invoke(
    {"image_desc": "A Neo-Classicism painting satirizing people looking at their smartphones."}
)

# Output the image prompt
print(image_prompt)

A Neo-Classicism painting depicting a grand, opulent setting reminiscent of classical art, featuring elegantly dressed figures in flowing robes and togas, gathered in a lush, sunlit garden. Each figure is engrossed in their smartphones, completely absorbed in their screens, creating a stark contrast between their traditional attire and modern technology. The lush greenery and marble statues serve as a backdrop, highlighting the absurdity of their distraction. Subtle expressions of amusement and bewilderment are visible on their faces as they interact with their devices, oblivious to the beauty around them. Incorporate classical architectural elements like columns and arches in the background to emphasize the Neo-Classical style. The color palette should be rich and vibrant, with soft, diffused lighting to enhance the dreamlike quality of the scene while maintaining a satirical tone.


Let’s use the previously generated image prompt as input to the `DallEAPIWrapper` to generate an image.

In [13]:
# Importing the DALL-E API Wrapper
from langchain_community.utilities.dalle_image_generator import DallEAPIWrapper
from IPython.display import Image

dalle = DallEAPIWrapper(
    model="dall-e-3",
    size="1024x1024",
    quality="standard",
    n=1,
)

# query
query = "A Neo-Classicism painting satirizing people looking at their smartphones."


# Generate image and retrieve URL
# Use chain.invoke() to convert the image description into a DALL-E prompt
# Use dalle.run() to generate the actual image
image_url = dalle.run(chain.invoke({"image_desc": query}))

# Display the generated image.
Image(url=image_url, width=500)

The image below was generated by DALL-E.

![dall-e_image.png](./assets/01-tools-dall-e.png)

## Custom Tools

In addition to the built-in tools provided by LangChain, you can define and use your own custom tools.

To do this, use the `@tool` decorator provided by the `langchain.tools` module to convert a function into a tool.

### @tool Decorator

This decorator allows you to transform a function into a tool. It provides various options to customize the behavior of the tool.

**How to Use**
1. Apply the `@tool` decorator above the function.
2. Set the decorator parameters as needed.

Using this decorator, you can easily convert regular Python functions into powerful tools, enabling automated documentation and flexible interface creation.

In [14]:
from langchain.tools import tool


# Convert a function into a tool using a decorator.
@tool
def add_numbers(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b


@tool
def multiply_numbers(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b

In [15]:
# Execute tool.
add_numbers.invoke({"a": 3, "b": 4})

7

In [16]:
# Execute tool.
multiply_numbers.invoke({"a": 3, "b": 4})

12

### Tavily Custom Tool: Enhancing Tool Control through Custom Tool Configuration


By using `@tool Decorator` , you can create a tool with improved control by leveraging the TavilyClient provided by the Tavily package.


Below are the key parameters used in the Tavily.

**Basic Search Configuration**

- query (str): The keyword or phrase to search for.
- search_depth (str): The level of detail for the search. Choose between **basic**  or **advanced** . (default: basic)
- topic (str): The topic area of the search. Choose between **general**  or **news** . (default: general)
- days (int): The recency of search results. Only results within the specified number of days will be returned. (default: 3)
- max_results (int): The maximum number of search results to retrieve. (default: 5)

**Domain Filtering**

- include_domains (list): A list of domains that must be included in the search results.
- exclude_domains (list): A list of domains to exclude from the search results.

**Detailed Result Settings**

- include_answer (bool): Whether to include answers generated by the API.
- include_raw_content (bool): Whether to include the original HTML content of the webpage.
- include_images (bool): Whether to include related image information.
- format_output (bool): Whether to apply formatting to the search results.

**Miscellaneous**

- kwargs : Additional keyword arguments. These may be used for future API updates or special features.

In [17]:
# %pip install tavily-python

In [18]:
# Import the TavilyClient class from the Tavily package.
from tavily import TavilyClient

tavily_tool = TavilyClient()

# Example of a search using various parameters
result1 = tavily_tool.search(
    query="Tell me about LangChain",  # Search query
    search_depth="advanced",  # Advanced search depth
    topic="general",  # General topic
    days=7,  # Results from the last 7 days
    max_results=10,  # Maximum of 10 results
    include_answer=True,  # Include answers
    include_raw_content=True,  # Include raw content
    include_images=True,  # Include images
    format_output=True,  # Format the output
)

# Print the results
print("Basic search results:", result1)

Basic search results: {'query': 'Tell me about LangChain', 'follow_up_questions': None, 'answer': "LangChain is an open-source framework designed to simplify the development of applications using large language models (LLMs) like OpenAI or Hugging Face. It enables the creation of dynamic, data-responsive applications that leverage the latest advancements in natural language processing. LangChain streamlines the development lifecycle of LLM applications, offering components and integrations for building stateful agents with streaming and human-in-the-loop support. It can be used for various applications such as chatbots, Generative Question-Answering (GQA), summarization, and more. LangChain's modular structure allows for chaining together different components to create advanced use cases around LLMs.", 'images': ['https://blog.enterprisedna.co/wp-content/uploads/2023/05/b0de1ae8-5b77-42c8-9be9-9daede50b378.png', 'https://substackcdn.com/image/fetch/w_1200,h_600,c_fill,f_jpg,q_auto:good

In [19]:
# Example of a news search.
result2 = tavily_tool.search(
    query="Latest AI technology trends",  # Search query
    search_depth="basic",  # Basic search depth
    topic="news",  # News topic
    days=3,  # Results from the last 3 days
    max_results=5,  # Maximum of 5 results
    include_answer=False,  # Exclude answers
    include_raw_content=False,  # Exclude raw content
    include_images=False,  # Exclude images
    format_output=True,  # Format the output
)

print("News search results:", result2)

News search results: {'query': 'Latest AI technology trends', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://techxplore.com/news/2025-01-lithium-sodium-ion-batteries-breakthroughs.html', 'title': 'Losing to lithium: Research shows sodium-ion batteries need breakthroughs to compete - Tech Xplore', 'score': 0.58976424, 'published_date': 'Mon, 13 Jan 2025 10:00:01 GMT', 'content': '##### Bias and discrimination in AI: Why sociolinguistics holds the key to better LLMs and a fairer world 5 hours ago ##### Sustainable cement: An electrochemical process to help neutralize cement industry CO₂ emissions Jan 11, 2025 ##### Smart glasses enter new era with sleeker designs, lower prices Jan 11, 2025 ##### Robots set to move beyond factory as AI advances Jan 10, 2025 ##### Light, flexible and radiation-resistant: Organic solar cells for space Jan 10, 2025 ##### Exploring quinone-based carbon capture: A promising path to safer CO₂ removal Jan 10, 2025 ##### M

In [20]:
# Example of a search with specific domain inclusion.
result3 = tavily_tool.search(
    query="Python programming tips",  # Search query
    search_depth="advanced",  # Advanced search depth
    max_results=3,  # Maximum of 3 results
    include_domains=["github.io"]
)

print("Search results with specific domain inclusion:", result3)

Search results with specific domain inclusion: {'query': 'Python programming tips', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Python Tips - williamkpchan.github.io', 'url': 'https://williamkpchan.github.io/LibDocs/Python+Tips.html', 'content': "In the past, we'd shared a list of Python programming tips for beginners that aimed to optimize code and reduce coding efforts. And our readers still enjoy reading it. So today, we're back with one more set of essential Python tips and tricks. All these tips can help you minify the code and optimize execution.", 'score': 0.7752821, 'raw_content': None}, {'title': "Python Tips | Ming's Blog - byrzhm.github.io", 'url': 'https://byrzhm.github.io/blog/posts/python-tips/', 'content': 'Python Tips. Posted Dec 20, 2024 . By Hongming Zhu. 1 min read. Python Tips. Contents. Python Tips ** Special Usages 1. Unpacking a dictionary into keyword arguments in a function call ... Programming Languages, Python. python. Th

In [21]:
# Example of a search excluding specific domains.
result4 = tavily_tool.search(
    query="Healthy diet",  # Search query
    search_depth="basic",  # Basic search depth
    days=30,  # Results from the last 30 days
    max_results=7,  # Maximum of 7 results
    exclude_domains=["ads.com", "spam.com"]
)

print("Search results excluding specific domains:", result4)

Search results excluding specific domains: {'query': 'Healthy diet', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Healthy diet - World Health Organization (WHO)', 'url': 'https://www.who.int/health-topics/healthy-diet', 'content': 'A healthy diet is a foundation for health, well-being, optimal growth and development. It protects against all forms of malnutrition. Unhealthy diet is one of the leading risks for the global burden of disease, mainly for noncommunicable diseases such as cardiovascular diseases, diabetes, and cancer.', 'score': 0.35555163, 'raw_content': None}, {'title': 'Healthy diet - World Health Organization (WHO)', 'url': 'https://www.who.int/initiatives/behealthy/healthy-diet', 'content': 'A healthy diet is essential for good health and nutrition. It protects you against many chronic noncommunicable diseases, such as heart disease, diabetes and cancer. Eating a variety of foods and consuming less salt, sugars and saturated and indus

In [22]:
# Define the tool.
@tool
def search_news(keyword: str) -> str:
    """Collect recent news for the given query. """
    tavily_client = TavilyClient()
    search_results = tavily_client.search(query=keyword, topic="news", days = 30)
    return search_results

print(search_news.invoke("AI Investment"))

{'query': 'AI Investment', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.jomfruland.net/surprising-stock-picks-uncover-hidden-opportunities-in-ai-stocks/', 'title': 'Surprising Stock Picks! Uncover Hidden Opportunities in AI Stocks. - Jomfruland.net', 'score': 0.6396691, 'published_date': 'Wed, 01 Jan 2025 14:02:16 GMT', 'content': 'Uncover Hidden Opportunities in AI Stocks. Uncover Hidden Opportunities in AI Stocks. Although AI stocks have reached soaring valuations, strategic investors can still find opportunities by identifying unexploited prospects in the market. Its strategic investment in OpenAI has integrated revolutionary AI technologies across Microsoft’s product suite, spurring growth and enhancing customer retention. Unleashing Investment Potential in AI: Exploring TSMC and Microsoft’s Strategic Moves Investor interest in AI technology remains strong, with TSMC and Microsoft poised as key players within the broader tech industry.

## Creating a Custom Tool for Google News Article Search

Define the `GoogleNews` class, which will be used as a tool to search for Google News articles.

**Note**
- No API key is required (because it uses RSS feeds).

This tool searches for news articles provided by **news.google.com** .

**Description**
- Uses the Google News search API to retrieve the latest news.
- Allows searching for news based on keywords.

**Key Parameters**
- `k` (int): Maximum number of search results to return (default: 5).

```python
# hl: Language, gl: Region, ceid: Region and Language Code
url = f"{self.base_url}?hl=en&gl=US&ceid=US:en" 
```

In the code, you can adjust the search results' language and region by modifying the language (hl), region (gl), and region and language code (ceid).

**Note**

Save the provided code as `google_news.py` , and then you can import it in other files using `from google_news import GoogleNews` .


In [23]:
#%pip install feedparser

In [24]:
import feedparser
from urllib.parse import quote
from typing import List, Dict, Optional


class GoogleNews:
    """
    This is a class for searching Google News and returning the results.
    """

    def __init__(self):
        """
        Initializes the GoogleNews class.
        Sets the base_url attribute.
        """
        self.base_url = "https://news.google.com/rss"

    def _fetch_news(self, url: str, k: int = 3) -> List[Dict[str, str]]:
        """
        Fetches news from the given URL.

        Args:
            url (str): The URL to fetch the news from.
            k (int): The maximum number of news articles to fetch (default: 3).

        Returns:
            List[Dict[str, str]]: A list of dictionaries containing news titles and links.
        """
        news_data = feedparser.parse(url)
        return [
            {"title": entry.title, "link": entry.link}
            for entry in news_data.entries[:k]
        ]

    def _collect_news(self, news_list: List[Dict[str, str]]) -> List[Dict[str, str]]:
        """
        Formats and returns the list of news articles.

        Args:
            news_list (List[Dict[str, str]]): A list of dictionaries containing news information.

        Returns:
            List[Dict[str, str]]: A list of dictionaries containing URLs and content.
        """
        if not news_list:
            print("No news available for the given keyword.")
            return []

        result = []
        for news in news_list:
            result.append({"url": news["link"], "content": news["title"]})

        return result

    def search_latest(self, k: int = 3) -> List[Dict[str, str]]:
        """
        Searches for the latest news.

        Args:
            k (int): The maximum number of news articles to search for (default: 3).

        Returns:
            List[Dict[str, str]]: A list of dictionaries containing URLs and content.
        """
        #url = f"{self.base_url}?hl=ko&gl=KR&ceid=KR:ko"
        url = f"{self.base_url}?hl=en&gl=US&ceid=US:en" # hl: 언어, gl: 지역, ceid: 지역 및 언어 코드
        news_list = self._fetch_news(url, k)
        return self._collect_news(news_list)

    def search_by_keyword(
        self, keyword: Optional[str] = None, k: int = 3
    ) -> List[Dict[str, str]]:
        """
        Searches for news using a keyword.  

        Args:
            keyword (Optional[str]): The keyword to search for (default: None).
            k (int): The maximum number of news articles to search for (default: 3).

        Returns:
            List[Dict[str, str]]: A list of dictionaries containing URLs and content.
        """
        if keyword:
            encoded_keyword = quote(keyword)
            url = f"{self.base_url}/search?q={encoded_keyword}&hl=en&gl=US&ceid=US:en"
        else:
            url = f"{self.base_url}?hl=en&gl=US&ceid=US:en"
        news_list = self._fetch_news(url, k)
        return self._collect_news(news_list)


In [25]:
google_tool = GoogleNews()

In [26]:
google_tool.search_by_keyword("AI Investment")

[{'url': 'https://news.google.com/rss/articles/CBMimAFBVV95cUxNaThyMnBjNjJpQjlQQTcwQjdoTEZPc1plbDdYU29YWlNVakE1NzVkV2ZNZHRyYlFSSjg1VDRNX2ZPT1NZdkM0ckVxMmxwaW1PbTdpay1EX2lUbFNqblIxOXdUZ3VDTVZoVmRlWVZXLTBUNG5SSUJZa3RieEFST0VjQ1hSRkxWNzRXSlBYb3k3QzEzUGdtNHo2dg?oc=5',
  'content': "A Once-in-a-Decade Investment Opportunity: 1 Artificial Intelligence (AI) Semiconductor Stock to Buy Hand Over Fist and Hold for the Next 10 Years (Hint: It's Not Nvidia) - The Motley Fool"},
 {'url': 'https://news.google.com/rss/articles/CBMipAFBVV95cUxNendoZ0U1eXVkVk5pYnljLThfWVotZENTbUJYZTVYRUhNOEFZNnlaYUZmT29FemhnQmJKRFptMml1cl9oQWdpM0NCQ3FhVkppcWlyeTNpUjdXOF9lNXQwR201eXM1UGxmazIzLTVLYVF2OEE1TVdBeHVwYk1uX0F3aHM0Q2p2R2R2S0p4eV8wZnFYVU9mdWJfd1VwS2I5RjVqTDZVQQ?oc=5',
  'content': 'AI Avatar Startup Synthesia Valued at $2.1 Billion - PYMNTS.com'},
 {'url': 'https://news.google.com/rss/articles/CBMie0FVX3lxTE9NbWttZW12S1BPLXJtM3Fhb1Z3NndndEZKRHo4Tk5sdzV3U0xvTFJHYmxjZ0lGUmNTRDcwLUlpZ3BCX0RrZmVvNDlXcFNVR3g4bEFlaTBj

In [27]:
from langchain.tools import tool
from typing import List, Dict


# Create a tool for searching news by keyword
@tool
def search_keyword(query: str) -> List[Dict[str, str]]:
    """Look up news by keyword"""
    print(query)
    news_tool = GoogleNews()
    return news_tool.search_by_keyword(query, k=5)

In [28]:
# Execution Results
search_keyword.invoke({"query": "LangChain AI"})

LangChain AI


[{'url': 'https://news.google.com/rss/articles/CBMid0FVX3lxTFBUVGIzM04zT1RYOC03QUxJenZLU3F4NVpoRmhvbFJRSnNZQVJIeUNyV2ktdERiSlplb2xQcXgyQWpHYS1DbEVRbk40alVBTzNQREFaQW40czNyNkdJcWxlQVgzdjVyZXI3UTN1aVVvYkdIS0tTU2lv?oc=5',
  'content': 'LangChain Unveils Innovative Ambient Agents for AI Interaction - Blockchain News'},
 {'url': 'https://news.google.com/rss/articles/CBMib0FVX3lxTE9qeUFQd08yN3pXenFYeWRTM1dHN3ZtTGVvaDAzREVNRU02ZVhNRFZCWmFpRm9obTN5QXF1TlpaYUI3d2VJYTBNeHNyUGJ3MXdVQWNDVW5MTVk4UDlfazFfX1hRRWpUeHBfUGRfS0pBdw?oc=5',
  'content': 'Vulnerabilities in LangChain Gen AI - Unit 42'},
 {'url': 'https://news.google.com/rss/articles/CBMixgFBVV95cUxNUzhFZEtsS2FrNTZ1ejVDejBQRThtSDA5TUtMbWRHOUxfUEtBVl8teERpZXp5U2wzbktZeWdTWnhKc2V4RnM2SXc0TzBsNVlQMktqUXRwNDliQW9yaTVnT3lzZ0lxeThobTliUEJRd05VM3V1anJ3bUh4cGJRbk44a1V0MVMzSEZfc2c4am04QnJBM2ZhZThTQ1NvYlhQUVBZbmhJWVZQdzlWWjE0UVdQRjYtWU1iSWl6WFBBTXdsQ2ROSHUxWmc?oc=5',
  'content': 'LangChain Meets Home Assistant: Unlock the Power of Generative AI in Yo